In [26]:

import pandas as pd
movies = pd.read_csv('movies.csv')
credits = pd.read_csv('credits.csv') 

In [27]:
movies = movies.merge(credits,on='title')

In [28]:
movies = movies[['movie_id','title','overview','production_companies','genres','keywords','cast','crew']]

In [29]:
import ast

In [30]:
def conv(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [31]:
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(conv)


In [32]:
movies['keywords'] = movies['keywords'].apply(conv)

In [33]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [34]:
movies['cast'] = movies['cast'].apply(convert3)

In [35]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [36]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [37]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [38]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [39]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [40]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [41]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])

In [42]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [44]:
vector = cv.fit_transform(new['tags']).toarray()

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
similarity = cosine_similarity(vector)

In [47]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [48]:
import pickle

In [49]:
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))